In [73]:
import pandas as pd
import numpy as np
from datetime import datetime

In [74]:
data_path='TestPort.csv'
trade_plan_path='ListTradeTran.xlsx'
trade_sheet='accY22StartJuly22'

asset_comm_path='AssetInfo.xlsx'
result_path=f'ReportTestPort_{trade_sheet}.xlsx'

In [75]:
TotalBudget=200000
# df=pd.read_csv('ASP-SME_KKP-SM-TRade.csv',index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df=pd.read_csv(data_path,parse_dates=['Date/Time'],dayfirst=True)
# df.index.set_names('date',inplace=True)
df.columns=['symbol','date','price']

list_symbols=df['symbol'].unique()
print(list_symbols)
df.info()
print(df.tail())

['ASP-SME' 'ASP-THEQ' 'KKP-ACT-EQ-A' 'KKP-SM-CAP' 'TISCOMS-A' 'TISCOFLEXP'
 'KFDYNAMIC' 'TSF-A']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5826 entries, 0 to 5825
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   symbol  5826 non-null   object        
 1   date    5826 non-null   datetime64[ns]
 2   price   5826 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 136.7+ KB
     symbol       date  price
5821  TSF-A 2022-12-30  84.53
5822  TSF-A 2023-01-03  85.13
5823  TSF-A 2023-01-04  83.97
5824  TSF-A 2023-01-05  83.92
5825  TSF-A 2023-01-06  84.42


In [76]:
dfPlan=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name=trade_sheet, \
                     parse_dates=['trade_date'],dtype ={'portion_pct': np.float64})
dfAssetInfo=pd.read_excel(asset_comm_path,engine="openpyxl",sheet_name='AssetInfo',\
                    index_col='symbol', dtype ={'buy_comm_pct': np.float64,'sell_comm_pct': np.float64})

tradeDateList= [ d for d in list(dfPlan['trade_date']) ]

print(tradeDateList)
print("================================================================================================")

print(dfPlan)
print(dfAssetInfo)


[Timestamp('2022-08-01 00:00:00'), Timestamp('2022-08-22 00:00:00'), Timestamp('2022-10-20 00:00:00'), Timestamp('2022-12-01 00:00:00'), Timestamp('2022-12-30 00:00:00'), Timestamp('2022-01-06 00:00:00')]
  trade_date  portion_pct action
0 2022-08-01         20.0      b
1 2022-08-22         20.0      b
2 2022-10-20         20.0      b
3 2022-12-01         20.0      b
4 2022-12-30         20.0      b
5 2022-01-06        100.0      s
              buy_comm_pct  sell_comm_pct
symbol                                   
ASP-SME               1.25            0.0
ASP-THEQ              1.00            0.0
KKP-SM-CAP            1.00            0.0
TISCOFLEXP            1.00            0.0
TSF-A                 1.05            0.0
TISCOMS-A             1.00            0.0
KFDYNAMIC             0.50            0.5
KKP-ACT-EQ-A          0.50            0.0


In [77]:
# veridf  there are both buy and sell if no sell , we have to add last reord 100 % as of last dat in Amibroker database

# day is ordering ? in plan

# verify sum buy =sum sell

# veriry all list in data price are in commision

In [78]:
def set_commision_by_action(item):
  x=item['symbol']  
  if item['action']=='b':
    comm_type='buy_comm_pct'
  elif  item['action']=='s':
    comm_type='sell_comm_pct'
  try:
     return dfAssetInfo.loc[x,comm_type]
  except Exception as ex:
     print(f"Error : no {x} to get commission")
     return 0
def get_qty_buy(item):
    xCost=(item['portion_pct']/100)*TotalBudget
    xPrice=item['price_on_comm']
    qty=xCost/xPrice
    return qty
def get_qty_sell(item,TotalQty):
    qty=(item['portion_pct']/100)*TotalQty
    return qty

In [79]:
dict_dfTran={}
dfSummary=pd.DataFrame(columns=['Symbol','BuyValue','AvgBuyPrice','SellValue','AvgSellPrice','PL','Percent_PL'])
for symbol in list_symbols:
    print(symbol)
    dfBS=df[ (df['symbol']==symbol) & (df['date'].isin(tradeDateList))]
    dfBS=dfBS.merge(dfPlan,how='inner',left_on='date',right_on='trade_date')
    
    dfBS['comm']=dfBS.apply(set_commision_by_action,axis=1)
    dfBS['price_on_comm']=dfBS.apply(lambda item : item['price']*(1+ (item['comm']/100)) if item['action']=='b' else item['price']*(1-(item['comm']/100)), axis=1)
    dfBS['price_on_comm']=dfBS['price_on_comm'].round(2)
    
    dfBS=dfBS.drop(columns=['trade_date'])
    #print(dfBS)
    
    print("================Buy=================")
    buyDF=dfBS.query("action=='b'")    
    # cal qury and actual value and  total_val and totl qury
    buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
    buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
    buyDF['action_value']=buyDF['value']
    print(buyDF[['date' ,'price_on_comm','qty','value','portion_pct']])
    
    TotalQty=buyDF['qty'].sum()
    TotalValue=round(buyDF['value'].sum(),2)
    AvgPrice=round(TotalValue/TotalQty,2)
    # print(f"Total Buy {symbol} : qty={TotalQty} and value={TotalValue} at AvgPrice={AvgPrice}")
    
    print("=================Sell=================")
    sellDF=dfBS.query("action=='s'")  
    sellDF['qty']=sellDF.apply(get_qty_sell,axis=1,args=(TotalQty,))
    sellDF['value']=sellDF['qty']*sellDF['price_on_comm']
    sellDF['action_value']=sellDF['value']*-1
    print(sellDF[['date' ,'price_on_comm','qty','value','portion_pct']])
    
    SellTotalQty=sellDF['qty'].sum()
    SellTotalValue=round(sellDF['value'].sum(),2)
    SellAvgPrice=round(SellTotalValue/SellTotalQty,2)
    
    # print(f"Total Sell {symbol} : qty={SellTotalQty} and value={SellTotalValue} at AvgPrice={SellAvgPrice}")
    
    PL=round(SellTotalValue-TotalValue,0)
    PCT_PL=round(PL/TotalValue*100,2)
    
    summary_data={'Symbol':symbol,'BuyValue':TotalValue,'AvgBuyPrice':AvgPrice,'SellValue':SellTotalValue,'AvgSellPrice':SellAvgPrice, \
                  'PL':PL,'Percent_PL':PCT_PL
                 }
    dfSummary=dfSummary.append(summary_data,ignore_index=True)
    
    dfTran=pd.concat([buyDF,sellDF],axis=0)
    dfTran=dfTran.sort_values(by=['date'])
    dfTran[['value','action_value']]=dfTran[['value','action_value']].round(0)
    
    dict_dfTran[symbol]=dfTran
    
    
    
    

    


ASP-SME
================Buy=================
        date  price_on_comm          qty    value  portion_pct
1 2022-08-01          17.00  2352.941176  40000.0         20.0
2 2022-08-22          17.30  2312.138728  40000.0         20.0
3 2022-10-20          17.79  2248.454188  40000.0         20.0
4 2022-12-01          17.80  2247.191011  40000.0         20.0
5 2022-12-30          17.85  2240.896359  40000.0         20.0
=================Sell=================
        date  price_on_comm           qty          value  portion_pct
0 2022-01-06          19.48  11401.621462  222103.586086        100.0
ASP-THEQ
================Buy=================
        date  price_on_comm          qty    value  portion_pct
1 2022-08-01          14.67  2726.653033  40000.0         20.0
2 2022-08-22          15.02  2663.115846  40000.0         20.0
3 2022-10-20          14.85  2693.602694  40000.0         20.0
4 2022-12-01          14.93  2679.169457  40000.0         20.0
5 2022-12-30          15.31  2612.671

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27316\3649325214.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27316\3649325214.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27316\3649325214.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

================Buy=================
        date  price_on_comm         qty    value  portion_pct
1 2022-08-01          59.17  676.018252  40000.0         20.0
2 2022-08-22          61.58  649.561546  40000.0         20.0
3 2022-10-20          59.61  671.028351  40000.0         20.0
4 2022-12-01          61.59  649.456081  40000.0         20.0
5 2022-12-30          62.30  642.054575  40000.0         20.0
=================Sell=================
        date  price_on_comm          qty          value  portion_pct
0 2022-01-06          64.94  3288.118805  213530.435169        100.0
KFDYNAMIC
================Buy=================
        date  price_on_comm          qty    value  portion_pct
1 2022-08-01          38.07  1050.696086  40000.0         20.0
2 2022-08-22          38.15  1048.492792  40000.0         20.0
3 2022-10-20          38.75  1032.258065  40000.0         20.0
4 2022-12-01          40.30   992.555831  40000.0         20.0
5 2022-12-30          41.33   967.819985  40000.0   

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27316\3649325214.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27316\3649325214.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27316\3649325214.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [80]:
writer=pd.ExcelWriter(result_path,engine='xlsxwriter') 
dfSummary=dfSummary.sort_values(by=['Percent_PL'],ascending=False)
dfSummary.to_excel(writer, sheet_name="Summary",index=False)
print(dfSummary)
for symbol,tranDF in dict_dfTran.items():
 tranDF.to_excel(writer, sheet_name=symbol,index=False) 
 #print(tranDF)
writer.save()

writer.close()

print("Successfully")


         Symbol  BuyValue  AvgBuyPrice  SellValue  AvgSellPrice       PL  \
4     TISCOMS-A  200000.0        20.50  222440.09         22.80  22440.0   
0       ASP-SME  200000.0        17.54  222103.59         19.48  22104.0   
5    TISCOFLEXP  200000.0        60.83  213530.44         64.94  13530.0   
3    KKP-SM-CAP  200000.0        17.42  213412.98         18.59  13413.0   
1      ASP-THEQ  200000.0        14.95  209723.33         15.68   9723.0   
7         TSF-A  200000.0        82.19  208593.13         85.72   8593.0   
2  KKP-ACT-EQ-A  200000.0        18.94  196355.31         18.59  -3645.0   
6     KFDYNAMIC  200000.0        39.28  192827.33         37.87  -7173.0   

   Percent_PL  
4       11.22  
0       11.05  
5        6.76  
3        6.71  
1        4.86  
7        4.30  
2       -1.82  
6       -3.59  


C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27316\1572397904.py:8: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()
D:\ProgramData\Anaconda3\envs\myquant_exp\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


Successfully
